In [21]:
# -*- coding: utf-8 -*-
import os
import torch
import gc
import pandas as pd
import chardet
from transformers import BertForQuestionAnswering, BertTokenizer

# Load the BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Professor's function: Extract answer from text
def answer_question(question, answer_text):
    input_ids = tokenizer.encode_plus(question, answer_text, truncation=True, max_length=512, return_tensors="pt")
    
    outputs = model(**input_ids)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    tokens = tokenizer.convert_ids_to_tokens(input_ids["input_ids"][0])
    if tokens[answer_start] in ["[CLS]", "[SEP]"]:
        return "No Answer Found"

    answer = tokens[answer_start]
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i].startswith("##"):
            answer += tokens[i][2:]
        else:
            answer += " " + tokens[i]

    return answer

# Function for computing probability of answer relevance
def answer_question_probability(question, answer_text):
    input_ids = tokenizer.encode_plus(question, answer_text, truncation=True, max_length=512, return_tensors="pt")

    outputs = model(**input_ids)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    probability = max(start_scores.tolist()[0]) + max(end_scores.tolist()[0])  # Combine scores

    return probability

# Function to determine the best section of long text
def answer_question_longfile_probability(question, answer_text, max_num_sentences, overlap_num_sentences):
    content = answer_text.splitlines()
    len_sentence = len(content)
    piece_num = - (- (len_sentence - max_num_sentences) // (max_num_sentences - overlap_num_sentences)) + 1
    prob = [0] * piece_num

    for jj in range(piece_num):
        start_idx = (max_num_sentences - overlap_num_sentences) * jj
        end_idx = min(start_idx + max_num_sentences, len_sentence)
        content_temp = " ".join(content[start_idx:end_idx])

        prob[jj] = answer_question_probability(question, content_temp)

    max_prob = max(prob)
    max_index = prob.index(max_prob)
    return max_index

# Improved function for handling long documents using a Sliding Window
def answer_question_longfile(question, answer_text, max_num_sentences, overlap_num_sentences):
    """
    Handles long documents by splitting into overlapping chunks (sliding window).
    """
    tokens = tokenizer.encode(answer_text, truncation=False)  # Get full tokenized text

    chunk_size = 400  # Each chunk has 400 tokens
    stride = 100  # Overlapping tokens for context retention

    chunks = [tokens[i: i + chunk_size] for i in range(0, len(tokens), chunk_size - stride)]

    best_answer = "No Answer Found"
    best_prob = float('-inf')

    for chunk in chunks:
        input_ids = tokenizer.encode_plus(question, chunk, truncation=True, max_length=512, return_tensors="pt")
        outputs = model(**input_ids)

        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        tokens_chunk = tokenizer.convert_ids_to_tokens(input_ids["input_ids"][0])

        if tokens_chunk[answer_start] in ["[CLS]", "[SEP]"]:
            continue  # Skip invalid answers

        answer = tokens_chunk[answer_start]
        for i in range(answer_start + 1, answer_end + 1):
            if tokens_chunk[i].startswith("##"):
                answer += tokens_chunk[i][2:]
            else:
                answer += " " + tokens_chunk[i]

        prob = max(start_scores.tolist()[0]) + max(end_scores.tolist()[0])
        if prob > best_prob:
            best_prob = prob
            best_answer = answer

    return best_answer

# Function to determine exclusive territory status
def determine_exclusive_territory(answer, full_text):
    exclusive_keywords = [
        "granted exclusive territory", "protected area", "exclusive rights", 
        "exclusive area", "territory is protected", "you have exclusive rights",
        "you are the sole operator", "no other franchisee", "defined geographic area","you are granted","exclusive territory granted"
    ]
    
    non_exclusive_keywords = [
        "you will not receive an exclusive territory", "does not grant", "We will not grant you a territory"," you do not receive any exclusive territory","You will not receive an exclusive Development Area.","You will not receive an exclusive"
        "no territorial rights", "not provided", "territory is non-exclusive", 
        "territory is shared", "territory may be reassigned"
    ]

    answer_lower = answer.lower()

    if any(keyword in answer_lower for keyword in exclusive_keywords):
        return "Yes"
    elif any(keyword in answer_lower for keyword in non_exclusive_keywords):
        return "No"

    full_text_lower = full_text.lower()
    if any(keyword in full_text_lower for keyword in exclusive_keywords):
        return "Yes"
    elif any(keyword in full_text_lower for keyword in non_exclusive_keywords):
        return "No"

    return "Uncertain"

# Function to clean extracted answers
def clean_answer(answer):
    answer = answer.replace(" - ", "-")
    answer = answer.replace(" ,", ",")
    answer = answer.replace(" .", ".")
    answer = answer.replace(" '", "'")
    answer = answer.capitalize()
    return answer

# Define the question for franchise agreements
question = "Does the franchisee is offered an exclusive territory."

# Parameters for long-text processing
max_num_sentences = 15
overlap_num_sentences = 7

# Path to dataset
dataset_path = "/USERS/kamalnadhkundla/desktop/codebert/dataset12"

# Process files in batches for memory efficiency
all_answers = []
batch_size = 10

for i in range(0, len(os.listdir(dataset_path)), batch_size):
    batch_files = os.listdir(dataset_path)[i:i + batch_size]

    for filename in batch_files:
        if filename.endswith(".txt"):
            file_path = os.path.join(dataset_path, filename)

            with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                answer_text = file.read()

            print(f"Processing: {filename}")

            # Extract answer using professor's method with sliding window fix
            raw_answer = answer_question_longfile(question, answer_text, max_num_sentences, overlap_num_sentences)
            cleaned_answer = "No Answer Found" if raw_answer in ["[CLS]", "[SEP]"] else clean_answer(raw_answer)

            # Determine Yes/No for Exclusive Territory
            exclusive_status = determine_exclusive_territory(cleaned_answer, answer_text)

            # Store result
            all_answers.append({
                "File Name": filename, 
                "Answer": cleaned_answer, 
                "Exclusive Territory (Yes/No)": exclusive_status
            })

    gc.collect()  # Free memory

# Convert results to DataFrame
df = pd.DataFrame(all_answers)



Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (2810 > 512). Running this sequence through the model will result in indexing errors


Processing: 20 Sonic Drive-In.pdf-all_items_item_12.txt
Processing: 52 Midas.pdf-all_items_item_12.txt
Processing: 19 Dairy Queen.pdf-all_items_item_12.txt
Processing: 5 Ace Hardware.pdf-all_items_item_12.txt
Processing: 42 Jimmy John's.pdf-all_items_item_12.txt
Processing: 8 Domino's.pdf-all_items_item_12.txt
Processing: 12 Pizza Hut.pdf-all_items_item_12.txt
Processing: 10 Taco Bell.pdf-all_items_item_12.txt
Processing: 25 Express Employment Professionals.pdf-all_items_item_12.txt
Processing: 33 IHOP.pdf-all_items_item_12.txt
Processing: 26 Chili's.pdf-all_items_item_12.txt
Processing: 21 Little Caesars.pdf-all_items_item_12.txt
Processing: 37 Culver's.pdf-all_items_item_12.txt
Processing: 18 Popeyes Louisiana Kitchen.pdf-all_items_item_12.txt
Processing: 3 KFC.pdf-all_items_item_12.txt
Processing: 43 Vavoline Instant Oil Change.pdf-all_items_item_12.txt
Processing: 39 Jersey Mikes.pdf-all_items_item_12.txt
Processing: 17 Panera Bread.pdf-all_items_item_12.txt
Processing: 35 Five Guy

In [22]:
supporting_statement=  {
    "1 McDonald's.pdf-all_items_item_12.txt": "The Franchise Agreement does not contain any exclusive grant, exclusive area, exclusive territorial rights, protected territory, or any right to exclude, control, or impose conditions on the location or development of future McDonald’s restaurants at any time. You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "3 KFC.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from Company-Owned Outlets, or from other channels of distribution or competitive brands that KFCLLC controls. However, so long as you are in compliance with the Franchise Agreement, you will have a protected territory of the smaller of (i) a radius of 1.5 miles of the Outlet, or (ii) an area around the Outlet where 30,000 people reside.",
    "4 Burger King.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "5 Ace Hardware.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other member stores, from stores that our affiliates own, competitive brands, and/or from other channels of distribution (for example, the internet and mail order houses).",
    "6 Chick-fil-A.pdf-all_items_item_12.txt": "You will not receive an exclusive or protected territory, express or implied. The License Agreement contains no exclusive grant, exclusive area, exclusive territorial rights, protected territory or right for you to exclude, disapprove, control or impose conditions on the location, development or operation of current or future Chick-fil-A Restaurants.",
    "7 Subway.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands we control.",
    "8 Domino's.pdf-all_items_item_12.txt": "You will not receive an exclusive territory under the Development Agreement. You may face competition from other franchisees, from outlets that we own or from other channels of distribution or competitive brands we control.",
    "9 Circle K.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "10 Taco Bell.pdf-all_items_item_12.txt": "You will not receive an exclusive territory under the License Agreement. You may face competition from other franchisees or licensees, from Units that we own, or from other channels of distribution or competitive brands that we or our affiliates control.",
    "11 Wendy's.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other Wendy’s franchisees, from company-owned Wendy’s Restaurants, or from other channels of distribution or competitive brands owned by Quality’s affiliates.",
    "12 Pizza Hut.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we (or our affiliates) own, or from other channels of distribution or competitive brands that we control. We will grant you a protected radius of 500 yards surrounding the Location of each System Restaurant (the “Protected Radius”).",
    "13 REMAX.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "14 Dunkin'.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You also do not have any type of nonexclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "15 Keller Williams.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control. However, during the term of the Franchise Agreement and for so long as you are in full compliance, we will not operate or authorize any other person to operate a bricks and mortar Market Center inside the Awarded Area.",
    "16 Tim Hortons.pdf-all_items_item_12.txt": "You will not receive exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "17 Panera Bread.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, and from other channels of distribution or competitive brands that we control. However, we will not operate a Panera Bread Bakery-Cafe or grant to a third party the right to operate a Panera Bread Bakery-Cafe within a geographic area (the 'Protected Area') consisting of a one (1) mile radius from the center of the Premises.",
    "18 Popeyes Louisiana Kitchen.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "19 Dairy Queen.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that ADQ owns, or from other channels of distribution or competitive brands that ADQ controls.",
    "20 Sonic Drive-In.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    
    "21 Little Caesars.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. We retain the right to establish additional locations nearby.",
    "22 Papa John's.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from Restaurants that we own, or from other channels of distribution or competitive brands that we control.",
    "23 Arby's.PDF-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "24 Applebees.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "25 Express Employment Professionals.pdf-all_items_item_12.txt": "Your Territory will be specified in the Franchise Agreement and will have a minimum of 30,000 jobs based on results from GbBIS, a division of Intelligent Direct Inc. As long as You perform Your duties under the Franchise Agreement, are not in default under the Franchise Agreement, and have maintained Your offices as required, We will not establish, except as otherwise described below, other third-party franchises or company-owned outlets selling or leasing the same products or services under the same or different trade name(s) or trademark(s) in Your Territory during the term of the Franchise Agreement.",
    "26 Chili's.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "27 Jack In The Box.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "28 Buffalo Wild Wings.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "29 Planet Fitness.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "30 Berkshire Hathaway.pdf-all_items_item_12.txt": "The Franchise Agreement does not grant you an exclusive territory. A protected territory may be provided by a written amendment to your Franchise Agreement. Some of the factors we will consider before granting a protected territory include your reputation in the marketplace, the size, market share, and expected Gross Revenues of your existing real estate business and the benefit of having you as a part of the System.",
    

   
     
    "31 The UPS Store.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "32 Servpro.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that We own, or from other channels of distribution or competitive brands that We control.",
    "33 IHOP.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "35 Five Guys.pdf-all_items_item_12.txt": "You will not receive an exclusive Territory under the Development Agreement. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "36 Denny's.pdf-all_items_item_12.txt": "Except for your right to operate a Denny’s restaurant at or from the premises of the restaurant, you do not receive any exclusive territory or the territorial rights under the franchise agreement.",
    "37 Culver's.pdf-all_items_item_12.txt": "Except as described above, you will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive franchise that we control. We may grant individual or multiple unit rights within the Designated Area to third parties.",
    "38 Wingstop.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "39 Jersey Mikes.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that Company owns, or from other channels of distribution or competitive brands that Company controls.",
    "40 Paris Baguette.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    
    
    "41 Hardee's.pdf-all_items_item_12.txt": "You will not receive any exclusive territory under the Franchise Agreement. You may face competition from other franchisees, from outlets that we own and/or operate, or from other channels of distribution or competitive brands that we control.",
    "42 Jimmy John's.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You also do not have any type of non-exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "43 Vavoline Instant Oil Change.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that VIOCF or its affiliates own, or from other channels of distribution or competitive brands that VIOCF controls.",
    "44 Home Instead.pdf-all_items_item_12.txt": "You are granted the right to operate a Franchised Business within a defined geographic area with an estimated minimum population of 10,000 people aged 65 years and older ('Protected Area'). We will not license the right to another franchisee to open and operate a Franchised Business nor will we operate a Home Instead business within the Protected Area granted to you by the Franchise Agreement.",
    "45 Zaxbys.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "46 Carl's Jr..pdf-all_items_item_12.txt": "You will not receive any exclusive territory under the Franchise Agreement. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "47 Baskin Robbins.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You also do not have any type of non-exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    "49 Snap-On Tools.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from us or other third parties or from other channels of distribution or competitive brands that we control.",
    "50 Anytime Fitness.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. You may face competition from other franchisees, from outlets that we own, or from other channels of distribution or competitive brands that we control.",
    
    



  "51 Jiffy Lube.pdf-all_items_item_12.txt": "Subject to any rights we have already granted to other franchisees, without your consent we will not (i) permit another franchisee to open a service center within three miles of your service center; or (ii) establish or operate a service center within three miles of your service center. Jiffy Lube can license another franchisee to open a service center or establish or operate a service center at any location outside of this three-mile ring.",
    "52 Midas.pdf-all_items_item_12.txt": "You will not receive an exclusive territory. The Franchise Agreement does not grant, confer or imply any geographic area, market, or exclusive territorial right to you. You may face competition from other franchisees, from outlets that we or our affiliates franchise or own, or from other channels of distribution or competitive brands that we control.",
    "54 Homevesters.pdf-all_items_item_12.txt": "You will not receive an exclusive Territory. We may establish other franchised or company-owned businesses that may compete in your Territory. You may face competition from other franchisees, from outlets that we or our affiliates own, or from other channels of distribution or competitive brands that we control."









}

   



In [23]:
original_decision={
    "1 McDonald's.pdf-all_items_item_12.txt": "No",
    "3 KFC.pdf-all_items_item_12.txt": "No",
    "4 Burger King.pdf-all_items_item_12.txt": "No",
    "5 Ace Hardware.pdf-all_items_item_12.txt": "No",
    "6 Chick-fil-A.pdf-all_items_item_12.txt": "No",
    "7 Subway.pdf-all_items_item_12.txt": "No",
    "8 Domino's.pdf-all_items_item_12.txt": "No",
    "9 Circle K.pdf-all_items_item_12.txt": "No",
    "10 Taco Bell.pdf-all_items_item_12.txt": "No",
    
    "11 Wendy's.pdf-all_items_item_12.txt": "No",
    "12 Pizza Hut.pdf-all_items_item_12.txt": "No",
    "13 REMAX.pdf-all_items_item_12.txt": "No",
    "14 Dunkin'.pdf-all_items_item_12.txt": "No",
    "15 Keller Williams.pdf-all_items_item_12.txt": "No",
    "16 Tim Hortons.pdf-all_items_item_12.txt": "No",
    "17 Panera Bread.pdf-all_items_item_12.txt": "No",
    "18 Popeyes Louisiana Kitchen.pdf-all_items_item_12.txt": "No",
    "19 Dairy Queen.pdf-all_items_item_12.txt": "No",
    "20 Sonic Drive-In.pdf-all_items_item_12.txt": "No",
    
    
    "21 Little Caesars.pdf-all_items_item_12.txt": "No",
    "22 Papa John's.pdf-all_items_item_12.txt": "No",
    "23 Arby's.PDF-all_items_item_12.txt": "No",
    "24 Applebees.pdf-all_items_item_12.txt": "No",
    "25 Express Employment Professionals.pdf-all_items_item_12.txt": "Yes",
    "26 Chili's.pdf-all_items_item_12.txt": "No",
    "27 Jack In The Box.pdf-all_items_item_12.txt": "No",
    "28 Buffalo Wild Wings.pdf-all_items_item_12.txt": "No",
    "29 Planet Fitness.pdf-all_items_item_12.txt": "No",
    "30 Berkshire Hathaway.pdf-all_items_item_12.txt": "No",
    
    
    "31 The UPS Store.pdf-all_items_item_12.txt": "No",
    "32 Servpro.pdf-all_items_item_12.txt": "No",
    "33 IHOP.pdf-all_items_item_12.txt": "No",
    "35 Five Guys.pdf-all_items_item_12.txt": "No",
    "36 Denny's.pdf-all_items_item_12.txt": "No",
    "37 Culver's.pdf-all_items_item_12.txt": "No",
    "38 Wingstop.pdf-all_items_item_12.txt": "No",
    "39 Jersey Mikes.pdf-all_items_item_12.txt": "No",
    "40 Paris Baguette.pdf-all_items_item_12.txt": "No",
    
    
    "41 Hardee's.pdf-all_items_item_12.txt": "No",
    "42 Jimmy John's.pdf-all_items_item_12.txt": "No",
    "43 Vavoline Instant Oil Change.pdf-all_items_item_12.txt": "No",
    "44 Home Instead.pdf-all_items_item_12.txt": "Yes",
    "45 Zaxbys.pdf-all_items_item_12.txt": "No",
    "46 Carl's Jr..pdf-all_items_item_12.txt": "No",
    "47 Baskin Robbins.pdf-all_items_item_12.txt": "No",
    "49 Snap-On Tools.pdf-all_items_item_12.txt": "No",
    "50 Anytime Fitness.pdf-all_items_item_12.txt": "No",
    
    
    "51 Jiffy Lube.pdf-all_items_item_12.txt": "Yes",
    "52 Midas.pdf-all_items_item_12.txt": "No",
    "54 Homevesters.pdf-all_items_item_12.txt": "No"


    
  
    
    


    
   





}

In [24]:
df

,File Name,Answer,Exclusive Territory (Yes/No)
0,20 Sonic Drive-In.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No
1,52 Midas.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No
2,19 Dairy Queen.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No
3,5 Ace Hardware.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No
4,42 Jimmy John's.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No
5,8 Domino's.pdf-all_items_item_12.txt,You will not receive an exclusive territory un...,No
6,12 Pizza Hut.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No
7,10 Taco Bell.pdf-all_items_item_12.txt,You will not receive an exclusive territory un...,No
8,25 Express Employment Professionals.pdf-all_it...,"Under the franchise agreement, you are granted...",Yes
9,33 IHOP.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No


In [25]:
df["Original Decision"] = df["File Name"].map(original_decision)
df["Support Statement"] = df["File Name"].map(supporting_statement)

In [26]:
df

,File Name,Answer,Exclusive Territory (Yes/No),Original Decision,Support Statement
0,20 Sonic Drive-In.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. Y...
1,52 Midas.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. T...
2,19 Dairy Queen.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. Y...
3,5 Ace Hardware.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. Y...
4,42 Jimmy John's.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. Y...
5,8 Domino's.pdf-all_items_item_12.txt,You will not receive an exclusive territory un...,No,No,You will not receive an exclusive territory un...
6,12 Pizza Hut.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. Y...
7,10 Taco Bell.pdf-all_items_item_12.txt,You will not receive an exclusive territory un...,No,No,You will not receive an exclusive territory un...
8,25 Express Employment Professionals.pdf-all_it...,"Under the franchise agreement, you are granted...",Yes,Yes,Your Territory will be specified in the Franch...
9,33 IHOP.pdf-all_items_item_12.txt,You will not receive an exclusive territory,No,No,You will not receive an exclusive territory. Y...


In [27]:
df.to_csv("12_dataframe.csv", index=False)
